## Create training datasets for decoder

In [ ]:
import sys
sys.path.append('../../')
import utils.mat_to_tfrecord
from pathlib import Path
import numpy as np

from T17_SessionArgs import get_session_info


sessions = [
    't17.2024.07.09',
    't17.2024.07.10',
    't17.2024.07.22',
     ]

participant = 't17'
bin_compression_factor = 2
channels_to_exclude = list(range(128,256)) #list(range(256, 384)) if ignoring hand-knob arrays
channels_to_zero = list(range(0,0)) #[] # leave empty to not zero anything

for session in sessions:
    
    trials_to_remove, block_nums, num_test_trials = get_session_info(session)
    session_path = str(Path('../Data', participant, session, 'Typing'))
    tfdata_path = str(Path(session_path, 'tfdata_20ms'))

    print(f'Sesison path: {session_path}')
    print(f'tfdata path: {session_path}')
    print('\n')

    args = {
        'session_mat_path': session_path,
        'block_nums': block_nums,
        'num_test_trials': num_test_trials,
        'trials_to_remove': trials_to_remove,
        'channels_to_exclude': channels_to_exclude,
        'channels_to_zero': channels_to_zero,
        'include_thresh_crossings': True,
        'include_spike_power': True,
        'spike_pow_max': 50000,
        'z_score_data': True,
        'global_std': True,
        'bin_compression_factor': bin_compression_factor,
        'save_path': tfdata_path,
    }

    utils.mat_to_tfrecord.main(args)

## Train the decoder. Remember to Restart the notebook first!

In [ ]:
from omegaconf import OmegaConf
import sys
sys.path.append('../')
from utils.brainToText_trainDecoder import brainToText_decoder

args = OmegaConf.load('T17_trainArgs.yaml')

decoder = brainToText_decoder(args)

infOut, stats = decoder.train()